In [2]:
thisUserId = 2


import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Movie4U") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/MovieLens.similarities") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/MovieLens.similarities") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/proxslick/.ivy2/cache
The jars for the packages stored in: /home/proxslick/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0017ca2f-ae10-4b03-be07-3c109ab9e811;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 160ms :: artifacts dl 6ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   arti

In [3]:
df = spark.read.format("mongo").load()
df.show()
df = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/MovieLens.ratings").load()
df.show()

+--------------------+---------+---------+------------------+
|                 _id|movieId_1|movieId_2|             value|
+--------------------+---------+---------+------------------+
|{61d3e7a574bf563e...|      307|     3424|0.1625990335790132|
|{61d3e7a574bf563e...|      307|     2134|0.0685758149545652|
|{61d3e7a574bf563e...|      307|      273|0.1128387743400316|
|{61d3e7a574bf563e...|      307|      288|0.1647382501186977|
|{61d3e7a574bf563e...|      307|       32|0.1978401970135654|
|{61d3e7a574bf563e...|      307|     2478|0.0681640217571773|
|{61d3e7a574bf563e...|      307|     2840|0.0479856672762316|
|{61d3e7a574bf563e...|      307|     3698| 0.066038399867317|
|{61d3e7a574bf563e...|      307|     3020|0.1014547108625799|
|{61d3e7a574bf563e...|      307|      208|0.0941508800193808|
|{61d3e7a574bf563e...|      307|     3893|0.0996330636401165|
|{61d3e7a574bf563e...|      307|      481|0.1377706460975944|
|{61d3e7a574bf563e...|      307|     1257| 0.087457284837197|
|{61d3e7

In [19]:
tableSimilarities = spark.read.format("mongo").load()
tableRatings = spark.read.format("mongo").option("uri", "mongodb://127.0.0.1/MovieLens.ratings").load()
userRatings = tableRatings.filter(tableRatings.userId == thisUserId)
userRatings.show()

+--------------------+-------+------+----------+------+
|                 _id|movieId|rating| timestamp|userId|
+--------------------+-------+------+----------+------+
|{61d3efde17f6bb06...|    170|   3.5|1192913581|     2|
|{61d3efde17f6bb06...|    849|   3.5|1192913537|     2|
|{61d3efde17f6bb06...|   1186|   3.5|1192913611|     2|
|{61d3efde17f6bb06...|   1235|   3.0|1192913585|     2|
|{61d3efde17f6bb06...|   1244|   3.0|1192913551|     2|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|
|{61d3efde17f6bb06...|   1663|   3.0|1192913565|     2|
|{61d3efde17f6bb06...|   1962|   4.0|1192913533|     2|
|{61d3efde17f6bb06...|   2108|   3.5|1192913541|     2|
|{61d3efde17f6bb06...|   2243|   4.5|1192913604|     2|
|{61d3efde17f6bb06...|   2352|   4.0|1192913575|     2|
|{61d3efde17f6bb06...|   2707|   3.5|1192913600|     2|
|{61d3efde17f6bb06...|   2746|   4.0|1192913548|     2|
|{61d3efde17f6bb06...|   2915|   3.5|1192913562|     2|
|{61d3efde17f6bb06...|   3363|   4.0|1192913596|

In [21]:
userRatingsJoined = userRatings.join(tableSimilarities, tableSimilarities["movieId_1"] == userRatings["movieId"])
userRatingsJoined.show()
userRatingsInterests = userRatingsJoined.withColumn("interests", (userRatings.movieId-3)*userRatingsJoined.value)

+--------------------+-------+------+----------+------+--------------------+---------+---------+------------------+
|                 _id|movieId|rating| timestamp|userId|                 _id|movieId_1|movieId_2|             value|
+--------------------+-------+------+----------+------+--------------------+---------+---------+------------------+
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e...|     1296|      170| 0.049263180104204|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e...|     1296|      849|0.0472721695592274|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e...|     1296|     1186|0.3110251399270678|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e...|     1296|     1235|0.2558560960208706|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e...|     1296|     1244| 0.284731283187942|
|{61d3efde17f6bb06...|   1296|   4.5|1192913608|     2|{61d3e7a774bf563e

In [28]:
userRatingsInterests.printSchema()
results = userRatingsInterests.orderBy(userRatingsInterests.interests.desc())
results.write.format("mongo").mode("append").option("database", "MovieLens").option("collection", "interests").save()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- movieId_1: integer (nullable = true)
 |-- movieId_2: integer (nullable = true)
 |-- value: double (nullable = true)
 |-- interests: double (nullable = true)



AttributeError: 'DataFrame' object has no attribute 'find'